Import the libraries needed

In [1]:
import pandas as pd
#from pandas.compat import StringIO
from pandas.tseries.offsets import DateOffset

from datetime import datetime
import psutil


import numpy as np
import os
import pyodbc
pyodbc.drivers()

import csv

from sklearn.preprocessing import StandardScaler
import seaborn as sns

pd.options.display.max_rows = 1000

# For the Yahoo Finance api
from pandas_datareader import data as pdr
import yfinance as yf
yf.pdr_override() 


C:\Users\m033\Anaconda3\lib\site-packages\requests\__init__.py:109: RequestsDependencyWarning: urllib3 (1.26.16) or chardet (3.0.0)/charset_normalizer (2.0.4) doesn't match a supported version!
  warnings.warn(


### Database connections and SQL to retrieve the various data needed.

In [2]:
SQL_server = 'xxx\yyy'
Reporting_database = 'zzz'

pd.set_option('max_row', None)
#pd.options.display.max_rows = 1000

# Set a value for the home folder.
home_folder = "."

# Set values for the various paths.
input_path = home_folder + "\data"

print('==========================')
now = datetime.now()
print(now)
print('==========================')
print(psutil.virtual_memory())
print('==========================')

2024-08-05 22:05:30.834639
svmem(total=16756752384, available=8869765120, percent=47.1, used=7886987264, free=8869765120)


# 1. Get supplementary policy features - Tax, Fees and Risk.

  -  Identified in Lit Review.

In [3]:
conn = pyodbc.connect(
         'DRIVER={ODBC Driver 17 for SQL Server};'
         'SERVER='+SQL_server+';'
         'DATABASE='+Reporting_database+';'
         'Trusted_Connection=yes;')

tax_query = f"""SELECT pol.Policy_Number         AS ContractNumber
                       ,CASE WHEN pol.Is_Lump_Sum_Taxation IS NULL THEN 0
                             ELSE pol.Is_Lump_Sum_Taxation 
                        END AS Lump_Sum_Taxation
                       ,pol.Tax_Notification_Type
                       ,pol.Insurance_Tax_Type
                       ,pol.Main_Tax_Country
                       ,pol.Secondary_Tax_Country
                       ,pol.Taxed_Retirement_Income
                       ,pol.Policy_Fee_Type_Desc
                       ,pol.Future_Risk
                 FROM CLT_Policy pol
                 INNER JOIN CLE_Reporting.dbo.CLT_Product prd ON prd.Product_Id = pol.Product_Id
                 WHERE prd.Product_Code IN ('FKP_2006V1','FKP_2006V2','FKP_2008V1','FKP_2010V1','FKP_2012V1','FKP_2017V1',
                                            'GNPP_2016V1','GNPP_2017V1','GNPP_2022V1','GNPP_2023V1',
                                            'GNRP_2008V1','GNRP_2010V1','GNRP_2012V1','GNRP_2017V1','GNRP_2022V1',
                                            'XRPP_2005V1','XRPP_2008V1','XRPP_2010V1','XRPP_2012V1','XRPP_2015V1')
                 ORDER BY pol.Policy_Number;"""

df_tax = pd.read_sql(tax_query, conn)

# Types and row count
# print stats.
print(len(df_tax.index))
now = datetime.now()
print(now)
print('==========================')

print(df_tax.dtypes)

# Turn off dislays to protect PII
#df_tax.head(5)

Error: ('HY000', '[HY000] [Microsoft][ODBC Driver 17 for SQL Server]SQL Server Network Interfaces: The system cannot contact a domain controller to service the authentication request. Please try again later.\r\n (-2146892976) (SQLDriverConnect); [HY000] [Microsoft][ODBC Driver 17 for SQL Server]Cannot generate SSPI context (-2146892976); [HY000] [Microsoft][ODBC Driver 17 for SQL Server]SQL Server Network Interfaces: The system cannot contact a domain controller to service the authentication request. Please try again later.\r\n (-2146892976); [HY000] [Microsoft][ODBC Driver 17 for SQL Server]Cannot generate SSPI context (-2146892976)')

## Drop obsolette or repeated features.

## Check for nulls

In [ ]:
df_tax.isna().any()

## Replace any nulls before checking the distributions of values for each column.

In [ ]:
df_tax['Tax_Notification_Type'] = df_tax['Tax_Notification_Type'].astype(str)
df_tax['Insurance_Tax_Type'] = df_tax['Insurance_Tax_Type'].fillna(0).apply(np.int64)
df_tax['Insurance_Tax_Type'] = df_tax['Insurance_Tax_Type'].fillna(0).apply(np.int64)



## Check the distributions of values for each column.

In [ ]:
#df_Horizon_All_Features['Lump_Sum_Taxation'] = df_Horizon_All_Features['Lump_Sum_Taxation'].fillna(0).apply(np.int64)
print('Lump_Sum_Taxation       : ',df_tax['Lump_Sum_Taxation'].value_counts())
print('Tax_Notification_Type   : ',df_tax['Tax_Notification_Type'].value_counts())
print('Insurance_Tax_Type      : ',df_tax['Insurance_Tax_Type'].value_counts())
print('Main_Tax_Country        : ',df_tax['Main_Tax_Country'].value_counts())
print('Secondary_Tax_Country   : ',df_tax['Secondary_Tax_Country'].value_counts())
print('Taxed_Retirement_Income : ',df_tax['Taxed_Retirement_Income'].value_counts())
print('Policy_Fee_Type_Desc    : ',df_tax['Policy_Fee_Type_Desc'].value_counts())
print('Future_Risk             : ',df_tax['Future_Risk'].value_counts())




## Policy_Fee_Type_Desc is the only feature showing a distribution across multiple values and is the only feature with potential predictive attributes.

## Drop all non predictive features and write to Excel

In [4]:
df_tax = df_tax.drop(['Lump_Sum_Taxation','Tax_Notification_Type','Insurance_Tax_Type','Main_Tax_Country','Secondary_Tax_Country','Taxed_Retirement_Income','Future_Risk'],axis=1)

# Write out to a comma separated values file.

# giving directory name
filename = input_path + '\Tax_Fee_Risk_data.csv'
df_tax.to_csv(filename, encoding='utf-8', index=False)        

# Turn off dislays to protect PII
#df_tax.head(10)

NameError: name 'df_tax' is not defined

In [5]:
%who_ls DataFrame 

[]

In [6]:
del df_tax



NameError: name 'df_tax' is not defined

In [7]:
print('==========================')
now = datetime.now()
print(now)
print('==========================')
print(psutil.virtual_memory())
print('==========================')

2024-08-05 22:06:57.768804
svmem(total=16756752384, available=8836472832, percent=47.3, used=7920279552, free=8836472832)
